In [ ]:
# ADK's Built-in Toolbox: The Foundation
# Welcome 👋🏻 This notebook demonstrates how to use Google's Agent Development Kit (ADK) to build an agent
# that leverages a built-in tool, specifically `google_search`, to fetch live event data.
# This approach allows agents to provide dynamic and tailored responses to user queries.
# Example: A user asks, “What food festivals are happening in London next month?”
# The agent will use the `google_search` tool to find the latest events and then summarize them.

In [ ]:
# Install Google ADK for Python
# This is the foundational package that provides all the necessary components for building and running your agents.
# The --quiet flag suppresses verbose output during installation.
%pip install google-adk --quiet

In [ ]:
# Verify ADK Installation (Optional but Recommended)
%pip show google-adk

In [ ]:
# Configure environment
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "False"

# Load API key from .env file
# Create a .env file with: GOOGLE_GENAI_API_KEY=your-api-key-here
api_key = os.environ.get("GOOGLE_GENAI_API_KEY")

if not api_key:
    raise ValueError(
        "GOOGLE_GENAI_API_KEY not found.\n"
        "Create a .env file with: GOOGLE_GENAI_API_KEY=your-api-key-here\n"
        "Get your API key from: https://aistudio.google.com/apikey"
    )

os.environ["GEMINI_API_KEY"] = api_key

print(f"✓ API key configured (length: {len(api_key)} characters)")
print(f"✓ GOOGLE_GENAI_USE_VERTEXAI = {os.environ.get('GOOGLE_GENAI_USE_VERTEXAI')}")

# Define the specific Gemini model we'll use. 
# 'gemini-2.0-flash' is a fast and efficient model.
MODEL_GEMINI_2_0_FLASH = "gemini-2.0-flash"

In [ ]:
# Import Necessary Modules

# Agent: The fundamental class for creating an AI agent within ADK.
from google.adk.agents import Agent

# Runner: The orchestrator that manages the flow of messages and agent execution.
from google.adk.runners import Runner

# InMemorySessionService: A simple, in-memory service for managing conversation sessions.
# Sessions are vital for maintaining context in multi-turn dialogues.
from google.adk.sessions import InMemorySessionService

# types from google.genai: Provides data structures (like Content and Part) to represent
# messages exchanged with the language model.
from google.genai import types

# google_search: The built-in tool that allows the agent to perform web searches.
from google.adk.tools import google_search

# Markdown, display: Used for rendering rich text output in Jupyter notebooks.
from IPython.display import Markdown, display

In [ ]:
# Modular Function for Agent Interaction
# This asynchronous function encapsulates the common logic for running an ADK agent interaction,
# making the main execution flow cleaner and more reusable.
async def run_adk_agent_interaction(
    agent: Agent,
    user_id: str,
    session_id: str,
    input_text: str,
    app_name: str = "default_app",
    session_service: InMemorySessionService = None,
) -> str:
    """
    Runs a single interaction with an ADK agent and returns its final text response.

    Args:
        agent: The ADK Agent instance to interact with.
        user_id: A unique identifier for the user initiating the interaction.
        session_id: A unique identifier for the conversation session.
                    Using a new ID for each distinct example ensures isolation.
        input_text: The textual message from the user.
        app_name: The name of the application using the agent. Defaults to "default_app".
        session_service: An optional InMemorySessionService instance. If None, a new
                         instance is created, making each call self-contained.

    Returns:
        The final text response from the agent as a string. Returns
        "No final response from agent." if no final response event is found.
    """
    # Use the provided session service if available, otherwise create a new one.
    # This flexibility allows for both isolated examples and continuous conversations.
    if session_service is None:
        session_service = InMemorySessionService()

    # Create a new session. In a multi-turn conversation, you would reuse
    # the same session_id across turns to maintain context.
    # For these distinct examples, we'll use unique session_ids.
    await session_service.create_session(app_name=app_name, user_id=user_id, session_id=session_id)

    # Format the user's input into a Content object, which is how messages
    # are structured for the agent.
    content = types.Content(role="user", parts=[types.Part(text=input_text)])

    # Initialize the Runner: This is the engine that connects the user's message
    # to the agent and manages the session state.
    runner = Runner(agent=agent, app_name=app_name, session_service=session_service)

    # Run the agent: This sends the user's message to the agent for processing.
    # The 'runner.run()' method returns an iterable of events that describe
    # the agent's actions and responses.
    events = runner.run(user_id=user_id, session_id=session_id, new_message=content)

    # Process the events to find and extract the final response from the agent.
    for event in events:
        if event.is_final_response():
            # Concatenate all parts in the final response
            response = ''.join(part.text for part in event.content.parts if part.text)
            return response
    return "No final response from agent."

print("Modular function `run_adk_agent_interaction` successfully defined.")

In [ ]:
# Latest Events Agent: Definition
# The instruction set is the core of the agent, guiding its behavior.
# It should be specific, structured, and ensure safe operation.
latest_events_agent = Agent(
    name="latest_events_agent",
    model=MODEL_GEMINI_2_0_FLASH,
    description="An agent that finds and summarizes current or upcoming events for a given destination and timeframe.",
    instruction="""
        You are a highly helpful and detailed travel assistant specialized in finding events.
        Your primary primary goal is to provide accurate, comprehensive, and well-organized information about events.

        When a user asks about events, follow these steps strictly:
        1.  **Identify Key Information:** Extract the desired event type (e.g., food festivals, music concerts, art exhibitions, sports events, cultural celebrations), destination, and timeframe (e.g., "next month", "August 2025", "next week", "soon").
        2.  **Use Google Search:** Utilize the `google_search` tool to find relevant current or upcoming events. Formulate precise search queries. For example, "food festivals London June 2025", "music concerts Berlin August 2025".
        3.  **Prioritize Official Sources:** Always aim to extract information from official event websites, reputable ticketing platforms, or well-known event listing sites.
        4.  **Summarize and Structure - IMPORTANT: FOLLOW THIS CLEANER FORMAT PRECISELY:**
            *   Start with a clear, friendly introduction.
            *   Present each event as a main item in a **Markdown unordered list (using asterisks `*`)**.
            *   Each event entry MUST follow this exact Markdown structure for clarity and spacing:
                ```
                *   **[Event Name]**
                    *   Description: [Brief Description].
                    *   Dates: [Start Date] - [End Date or Single Date].
                    *   Location: [Venue/Area], [City].
                    *   [More Info](URL) (Optional: if official URL found)
                ```
            *   Ensure only the "Event Name" is bolded as a main heading for each event.
            *   If dates are a range, use "Start Date - End Date". If single day, use "Single Date".
            *   Organize the events **chronologically by start date** if multiple events are found.

            **Example Event Formatting (pay close attention to line breaks and reduced bolding):**
            ```
            *   **Taste of London**
                *   Description: A renowned food festival featuring top restaurants and culinary experiences.
                *   Dates: June 18 - June 22, 2025.
                *   Location: Regent's Park, London.
                *   [More Info](https://www.tasteoflondon.co.uk/)

            *   **Berlin Lollapalooza**
                *   Description: A major music festival with diverse artists and stages.
                *   Dates: August 9 - August 10, 2025.
                *   Location: Olympiapark and Olympiastadion, Berlin.
                *   [More Info](https://www.lollapaloozade.com/)
            ```

        5.  **Handle Missing Information:**
            *   If the timeframe is vague (e.g., "soon", "what's happening?"), default to searching for events in the *upcoming 1-2 months* from the current date. If you find too many, suggest a more specific timeframe.
            *   If no specific event type is requested, search for a broad range of popular events (e.g., "major events [city] [month]").
        6.  **No Results Found:** If, after searching, no relevant events are found, politely state that no events matching the criteria could be found and offer to search for different dates or event types.
        7.  **Maintain Tone:** Be concise, friendly, and helpful.
""",
    tools=[google_search], # Assign the google_search tool to this agent.
)

print("`latest_events_agent` successfully defined with comprehensive instructions.")

In [ ]:
# Define common parameters for our agent interactions.
APP_NAME = "wanderwise_events_app"
USER_ID = "user_001"

# Create a common session service instance to reuse across multiple interactions.
common_session_service = InMemorySessionService()

# Define a dictionary of user inputs for various test cases.
# Each entry includes a descriptive name and the user's query.
EVENT_TEST_CASES = {
    "Normal Case: Food Festivals, London, Next Month (July 2025)":
        "What food festivals are happening in London next month?",

    "Specific Case: Music Concerts, Berlin, August 2025":
        "Music concerts in Berlin in August 2025.",

    "Vague Timeframe: General Events, Tokyo, Soon":
        "What's happening in Tokyo soon?",

    "Multiple Event Types: Cultural Events & Art Exhibitions, Paris, October 2025":
        "What are the major cultural events and art exhibitions happening in Paris in October 2025?",

    "Specific Event Type: Major Sports Events, New York City, July 2025":
        "Any major sports events in New York City in July 2025?",

    "No Results Expected: UFO Conventions, Hypothetical Location, Next Week (June 2025)":
        "Are there any major UFO conventions in Garetwasterland next week?",

    "Past Date Request: Festivals, Rio, January 2024":
        "What festivals happened in Rio in January 2024?", # Should clarify or state past

    "Future Date Request: Tech Conferences, San Francisco, December 2026":
        "Any big tech conferences in San Francisco in December 2026?",
}

print("------ [starting agent interactions] ------\n")

# Loop through each test case and run the agent.
for case_name, user_input in EVENT_TEST_CASES.items():
    # Create a unique session ID for each interaction to ensure isolated tests.
    session_id = f"{APP_NAME}_session_{case_name.replace(' ', '_').replace(',', '').lower()}"

    print(f"\n--- Testing >> {case_name} ---")
    print(f"User Input: '{user_input}'")

    # Invoke the latest_events_agent using the modular helper function.
    agent_response = await run_adk_agent_interaction(
        agent=latest_events_agent,
        user_id=USER_ID,
        session_id=session_id,
        input_text=user_input,
        app_name=APP_NAME,
        session_service=common_session_service # Pass the common session service
    )

    # Display the agent's response.
    display(Markdown(f"**Agent Response:**\n{agent_response}"))
    print(f"\n--- Testing Complete >> {case_name} ---")


print("\n------ [all agent interactions complete] ------")

In [ ]:
# Congratulations 🎉 
# You’ve just built and run an agent that uses the ADK built-in `google_search` tool
# to fetch live event data for a travel itinerary.
# This demonstrates the power of combining LLM reasoning with external tools to create
# truly intelligent and practical applications.
# Try modifying the prompt or agent instructions, and see how the results change!